<a href="https://colab.research.google.com/github/sujit0892/AI-Generated-Text-Detection/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [4]:
!pip install textstat

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import spacy
from textstat import textstat
import math
from nltk import ngrams
from collections import Counter

## Read Data

In [6]:
#shuffle the data
df = pd.read_csv('train.csv').sample(frac=1)

In [7]:
display(df.head())
display(df.info())
display(df.isna().sum())

,text,target
35354,SEE FULL RESULTS AND MAPS FROM THE NEW YORK PR...,llama-8B
18531,Title: Jeb Bush Advocates for Global Coalitio...,mistral-7B
7738,"""Where Are U Now"": Inside the Studio with Ju...",gemma-2-9b
40827,Roger Federer Introduces Innovative Twist on C...,accounts/yi-01-ai/models/yi-large
2985,The New York Times\nPaul Krugman|Paul Ryan Is ...,Human_story


<class 'pandas.core.frame.DataFrame'>
Index: 51247 entries, 35354 to 47588
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    51181 non-null  object
 1   target  51247 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


None

,0
text,66
target,0


In [8]:
df.dropna(inplace=True)

In [9]:
# train test split
df_train, df_test, _ , _ = train_test_split(df,df['target'],test_size=0.2,random_state=42,stratify=df['target'])

In [10]:
#train
display(df_train.shape)
display(df_train.head())
display(df_train.info())
display(df_train.target.value_counts(normalize=True))

(40944, 2)

,text,target
14312,House Votes to Remove Confederate Statues fro...,gemma-2-9b
20037,Title: Embracing the Therapeutic Power of Puz...,mistral-7B
48151,Jil Sander's Fall 2016 Collection: A Mastercla...,GPT_4-o
12368,We Knew Our Foster Son Would Be With Us All S...,gemma-2-9b
30304,"""En Garde Arts Brings Big Outdoor Site Specifi...",llama-8B


<class 'pandas.core.frame.DataFrame'>
Index: 40944 entries, 14312 to 778
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40944 non-null  object
 1   target  40944 non-null  object
dtypes: object(2)
memory usage: 959.6+ KB


None

,proportion
target,
GPT_4-o,0.143025
accounts/yi-01-ai/models/yi-large,0.143000
mistral-7B,0.142951
qwen-2-72B,0.142903
gemma-2-9b,0.142829
llama-8B,0.142756
Human_story,0.142536


In [11]:
#test
display(df_test.shape)
display(df_test.head())
display(df_test.info())
display(df_test.target.value_counts(normalize=True))

(10237, 2)

,text,target
44116,Title: When Sleep Takes Center Stage: A Holida...,GPT_4-o
5765,"Supported by\nRob Burrow, Rugby Star and A.L.S...",Human_story
21602,"Title: ""Natasha Tretheway's Selection for Poe...",mistral-7B
22756,"In the world of long-distance athletics, runni...",qwen-2-72B
48088,"The Latest Data on Coronavirus in Atchison, K...",GPT_4-o


<class 'pandas.core.frame.DataFrame'>
Index: 10237 entries, 44116 to 33818
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10237 non-null  object
 1   target  10237 non-null  object
dtypes: object(2)
memory usage: 239.9+ KB


None

,proportion
target,
GPT_4-o,0.143108
accounts/yi-01-ai/models/yi-large,0.143011
mistral-7B,0.142913
qwen-2-72B,0.142913
gemma-2-9b,0.142815
llama-8B,0.142718
Human_story,0.142522


## NLP Feature Engineering

In [12]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
#extract lexical feature
def extract_lexical_feature(text):
  words = nltk.word_tokenize(text)
  sentances =nltk.sent_tokenize(text)

  num_words = len(words)
  num_sentances = len(sentances)
  avg_word_length = sum(len(word) for word in words) / num_words
  avg_sentance_length = sum(len(sentance) for sentance in sentances) / num_sentances
  type_token_ratio = len(set(words)) / num_words
  return {"num_words":num_words,
          "num_sentances":num_sentances,
          "avg_word_length":avg_word_length,
          "avg_sentance_length":avg_sentance_length,
          "type_token_ratio":type_token_ratio
          }


In [14]:
 df_train = df_train.join(df_train['text'].apply(extract_lexical_feature).apply(pd.Series))

In [15]:

#extract stylistic features
def extract_semantic_feature(text):
  doc = nlp(text)
  sentances = list(doc.sents)
  words = [token.text.lower() for token in doc if not token.is_alpha]
  repetitivness = len(words) -len(set(words))

  if len(sentances) > 1:
    coherence = sum(sentances[i].similarity(sentances[i+1]) for i in range(len(sentances)-1)) / len(sentances)-1
  else:
    coherence = 0

  return {"repetitivness":repetitivness,
          "coherence":coherence
          }

In [16]:
 df_train = df_train.join(df_train['text'].apply(extract_semantic_feature).apply(pd.Series))

<ipython-input-15-27765a3dd7a5>:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  coherence = sum(sentances[i].similarity(sentances[i+1]) for i in range(len(sentances)-1)) / len(sentances)-1


In [17]:
#extract syntatic features
def extract_stylistic_features(text):
  readability = textstat.flesch_reading_ease(text)
  fullstops_count = text.count('.')
  question_marks_count = text.count('?')
  exclamation_marks_count = text.count('!')
  comma_counts = text.count(',')
  semi_colon_counts = text.count(';')
  colon_counts = text.count(':')
  return {"readability":readability,
          "fullstops_count":fullstops_count,
          "question_marks_count":question_marks_count,
          "exclamation_marks_count":exclamation_marks_count,
          "comma_count":comma_counts,
          "semi_colon_count":semi_colon_counts,
          "colon_count":colon_counts
          }


In [18]:
 df_train = df_train.join(df_train['text'].apply(extract_stylistic_features).apply(pd.Series))

In [21]:

# extract syntactic features
def extract_syntatic_feature(text):
  doc = nlp(text)
  pos_tags = Counter([token.pos_ for token in doc])

  return pos_tags


In [22]:
 df_train = df_train.join(df_train['text'].apply(extract_syntatic_feature).apply(pd.Series))

In [23]:
df_train.head()

,text,target,num_words,num_sentances,avg_word_length,avg_sentance_length,type_token_ratio,repetitivness,coherence,readability,...,NOUN,ADV,CCONJ,PRON,AUX,SCONJ,NUM,X,INTJ,SYM
14312,House Votes to Remove Confederate Statues fro...,gemma-2-9b,410.0,14.0,4.795122,166.000000,0.502439,41.0,-0.468025,49.15,...,74.0,11.0,16.0,20.0,17.0,7.0,2.0,NaN,NaN,NaN
20037,Title: Embracing the Therapeutic Power of Puz...,mistral-7B,447.0,18.0,5.000000,144.777778,0.512304,67.0,-0.506253,32.53,...,110.0,10.0,16.0,27.0,16.0,6.0,1.0,NaN,NaN,NaN
48151,Jil Sander's Fall 2016 Collection: A Mastercla...,GPT_4-o,609.0,29.0,5.095238,124.172414,0.553366,99.0,-0.621676,32.43,...,157.0,28.0,23.0,23.0,11.0,9.0,8.0,7.0,NaN,NaN
12368,We Knew Our Foster Son Would Be With Us All S...,gemma-2-9b,447.0,25.0,4.082774,86.520000,0.510067,74.0,-0.665558,64.71,...,84.0,15.0,17.0,41.0,29.0,7.0,2.0,NaN,NaN,NaN
30304,"""En Garde Arts Brings Big Outdoor Site Specifi...",llama-8B,450.0,14.0,4.835556,181.285714,0.448889,69.0,-0.445844,34.70,...,98.0,7.0,24.0,16.0,14.0,2.0,1.0,NaN,1.0,NaN


In [24]:
df_train.fillna(0,inplace=True)


In [25]:
df_train.to_csv('syntatical_train_features.csv',index=False)

In [26]:
df_test = df_test.join(df_test['text'].apply(extract_lexical_feature).apply(pd.Series))
df_test = df_test.join(df_test['text'].apply(extract_semantic_feature).apply(pd.Series))
df_test = df_test.join(df_test['text'].apply(extract_stylistic_features).apply(pd.Series))
df_test = df_test.join(df_test['text'].apply(extract_syntatic_feature).apply(pd.Series))
df_test.fillna(0,inplace=True)
df+test.to_csv('syntatical_test_features.csv',index=False)

<ipython-input-15-27765a3dd7a5>:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  coherence = sum(sentances[i].similarity(sentances[i+1]) for i in range(len(sentances)-1)) / len(sentances)-1


In [27]:
df_test.to_csv('syntatical_test_features.csv',index=False)